In [1]:
# 使用submitter对api进行访问
from sparksampling import Submitter
from sparksampling.var import FILE_TYPE_CSV
from sparksampling.var import SIMPLE_RANDOM_SAMPLING_METHOD
submitter = Submitter()
dataset_uri = 'hdfs://localhost:9000/dataset/ten_million_top1k.csv'
fraction = 0.1

In [2]:
# 提交抽样任务
submit_response = submitter.submit_sampling_simplejob(dataset_uri,
                                              method=SIMPLE_RANDOM_SAMPLING_METHOD,
                                              file_type=FILE_TYPE_CSV,
                                              fraction=fraction,
                                              with_header=True)
job_id = submit_response.job_id
submit_response.to_dict()

2021-05-29 16:46:58,049 - INFO - request: http://localhost:8000/v1/sampling/simplejob/ with data {'path': 'hdfs://localhost:9000/dataset/ten_million_top1k.csv', 'method': 'random', 'type': 'csv', 'with_header': True, 'conf': {'fraction': 0.1}}


{'code': 0, 'msg': '', 'data': {'job_id': 10085}}

In [3]:
# 查询抽样任务
sampling_job_details = submitter.get_sampling_job_details(job_id)
sampled_path = sampling_job_details.sampled_path
print(sampling_job_details.to_dict())
sampled_path

2021-05-29 16:43:47,327 - INFO - request: http://localhost:8000/v1/query/sampling/job/ with data {'job_id': 10084}


{'code': 0, 'msg': '', 'data': {'job_id': 10084, 'job_status': 'Succeed', 'msg': 'succeed', 'method': 'Simple Random Sampling', 'start_time': '2021/05/29/ 16:43:41', 'end_time': '2021/05/29 16:43:45', 'simpled_file_path': 'hdfs://localhost:9000/dataset/ten_million_top1k.csv-sampled-1622277821.4077992', 'request_data': "{'path': 'hdfs://localhost:9000/dataset/ten_million_top1k.csv', 'method': 'random', 'type': 'csv', 'with_header': True, 'conf': {'fraction': 0.1, 'path': 'hdfs://localhost:9000/dataset/ten_million_top1k.csv', 'method': 'random', 'file_type': 'csv', 'with_header': True, 'seed': 55626, 'with_replacement': True, 'col_key': None}}"}}


'hdfs://localhost:9000/dataset/ten_million_top1k.csv-sampled-1622277821.4077992'

In [4]:
# 简单的读取抽样之后的文件
from pyspark.sql import SparkSession
from sparksampling.config import SPARK_CONF

conf = SPARK_CONF
spark = SparkSession.builder.config(conf=conf).getOrCreate()
df = spark.read.csv(sampled_path, header=True).toPandas()
# 可以在这后面做数据分析，或试试看下面的统计、评估功能
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()

train_y = df['y']
train_X = df[['X_20','X_80']]
# train_X = df.drop(["# id"], axis=1)
model.fit(train_X,train_y)
tsdf = spark.read.csv(dataset_uri, header=True)
tdf = tsdf.toPandas()

test_y = tdf['y']
test_X = tdf[train_X.columns]
# test_X = test_X[feature_list]
pred_y = model.predict(test_X)
# data analyse here
from sklearn.metrics import classification_report
print(classification_report(y_true=test_y, y_pred=pred_y))

              precision    recall  f1-score   support

           0       0.88      0.89      0.88       517
           1       0.88      0.87      0.87       483

    accuracy                           0.88      1000
   macro avg       0.88      0.88      0.88      1000
weighted avg       0.88      0.88      0.88      1000



In [5]:
# 统计原数据集
submitter.get_statistics(path=dataset_uri, file_type=FILE_TYPE_CSV,with_header=True).to_pandas()

2021-05-29 16:44:05,141 - INFO - request: http://localhost:8000/v1/evaluation/statistics/ with data {'path': 'hdfs://localhost:9000/dataset/ten_million_top1k.csv', 'type': 'csv', 'method': 'basic', 'with_header': True, 'from_sampling': False}


,summary,# id,X_0,X_1,X_2,X_3,X_4,X_5,X_6,X_7,...,X_91,X_92,X_93,X_94,X_95,X_96,X_97,X_98,X_99,y
0,count,1000,1000,1000,1000,1000,1000,1000,1000,1000,...,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000
1,mean,499.5,2.049,3.051,2.815,3.118,4.211,3.992,3.351,2.658,...,0.029485678112799993,-0.015578821739792,-0.034382649559500045,0.03410696849619999,0.004251966935300004,0.031006250621699986,-0.008798382997400003,0.009004523887710001,-0.05045604444349996,0.483
2,stddev,288.8194360957494,1.8889592815005751,2.394186945735675,2.0559674126799785,1.863208938242715,1.6208350027396687,1.5317579058984108,1.7728344372619638,1.9335911601925522,...,1.310014162316104,0.9928149233047021,1.3489092513130678,1.0175306226012728,0.9937576399485477,1.2158726897277283,1.4720581081411457,0.8531823148864874,1.0285753546436174,0.4999609594367951
3,min,0,0,0,0,0,1,2,1,0,...,-0.0021861139000000003,-0.00099003483,-0.007215624499999999,-0.0087255505,-0.0049381842,-0.0030899136,-0.0012198847,-0.0017179298999999999,-0.00151381,0
4,max,999,6,6,6,6,6,6,6,6,...,4.247343099999999,3.266212,3.7208654,3.1033252,3.1775257999999997,3.5107522000000007,5.6920133,3.0476226000000004,3.1554324,1


In [6]:
# 统计抽样后的数据集
data = submitter.get_statistics(job_id=job_id, from_sampling=True, file_type=FILE_TYPE_CSV,with_header=True).to_pandas()
data

2021-05-29 16:44:09,887 - INFO - request: http://localhost:8000/v1/evaluation/statistics/ with data {'job_id': 10084, 'type': 'csv', 'method': 'basic', 'with_header': True, 'from_sampling': True}


,summary,# id,X_0,X_1,X_2,X_3,X_4,X_5,X_6,X_7,...,X_91,X_92,X_93,X_94,X_95,X_96,X_97,X_98,X_99,y
0,count,93,93,93,93,93,93,93,93,93,...,93,93,93,93,93,93,93,93,93,93
1,mean,494.5376344086022,1.7204301075268817,3.10752688172043,2.7419354838709675,3.247311827956989,4.021505376344086,3.817204301075269,3.3978494623655915,2.4301075268817205,...,0.10275933353763449,0.06457782311827955,-0.07502768804301073,-0.049317472182795676,0.06376470226881722,0.06748917784946236,-0.1938171594301075,0.10569752317741932,-0.10600688911827955,0.5161290322580645
2,stddev,273.9209882564435,1.7654017365057724,2.5429457652554026,2.0950418995833515,1.8976868870473917,1.7002461114923015,1.4666305405895497,1.745427473156253,2.012989608076837,...,1.2772515681097716,0.9118345726290845,1.3217570100028078,1.08458308962452,1.0567088752154805,1.1914036886669042,1.340142254040861,0.7929760877804981,0.9064572502890808,0.502448423176264
3,min,116,0,0,0,0,1,2,1,0,...,-0.15532976,-0.017350986000000002,-0.037803247000000005,-0.015101419,-0.020774542,-0.023479291,-0.014307641000000001,-0.021572345,-0.00151381,0
4,max,971,6,6,6,6,6,6,6,6,...,2.7390996,2.0388664,3.7208654,2.1955156000000002,2.7141035,2.2594397999999996,3.4823716,1.9738924999999998,2.2422427000000003,1


In [7]:
# 提交评估任务
cmp_evaluation_job = submitter.submit_evaluation_job(compare_job_id=job_id, file_type=FILE_TYPE_CSV)
print(cmp_evaluation_job.to_dict())
cmp_evaluation_job_id = cmp_evaluation_job.job_id
cmp_evaluation_job_id

2021-05-29 16:44:13,127 - INFO - request: http://localhost:8000/v1/evaluation/job/ with data {'method': 'compare', 'type': 'csv', 'compare_job_id': 10084}


{'code': 0, 'msg': '', 'data': {'job_id': 50020}}


50020

In [8]:
# 查看各属性评估得分
cmp_evaluation_job_data = submitter.get_evaluation_job_details(job_id=cmp_evaluation_job_id)
cmp_df = cmp_evaluation_job_data.to_pandas()
cmp_df

2021-05-29 16:44:17,240 - INFO - request: http://localhost:8000/v1/query/evaluation/job/ with data {'job_id': 50020}


,# id,X_0,X_1,X_10,X_11,X_12,X_13,X_14,X_15,X_16,...,X_91,X_92,X_93,X_94,X_95,X_96,X_97,X_98,X_99,y
count,93,93,93,93,93,93,93,93,93,93,...,93,93,93,93,93,93,93,93,93,93
mean,494.5376344086022,1.7204301075268817,3.10752688172043,2.2795698924731185,3.053763440860215,3.193548387096774,2.21505376344086,2.4516129032258065,-0.14672708488172037,-0.1938171594301075,...,0.10275933353763449,0.06457782311827955,-0.07502768804301073,-0.049317472182795676,0.06376470226881722,0.06748917784946236,-0.1938171594301075,0.10569752317741932,-0.10600688911827955,0.5161290322580645
stddev,273.9209882564435,1.7654017365057724,2.5429457652554026,1.7217634501330656,1.7959228667822873,2.11237587500595,1.9158915643137293,1.6319908423140475,0.9740645859553754,1.340142254040861,...,1.2772515681097716,0.9118345726290845,1.3217570100028078,1.08458308962452,1.0567088752154805,1.1914036886669042,1.340142254040861,0.7929760877804981,0.9064572502890808,0.502448423176264
min,116,0,0,0,1,0,0,0,-0.05648472,-0.014307641000000001,...,-0.15532976,-0.017350986000000002,-0.037803247000000005,-0.015101419,-0.020774542,-0.023479291,-0.014307641000000001,-0.021572345,-0.00151381,0
max,971,6,6,5,6,6,6,5,3.0338942,3.4823716,...,2.7390996,2.0388664,3.7208654,2.1955156000000002,2.7141035,2.2594397999999996,3.4823716,1.9738924999999998,2.2422427000000003,1
mean_bias,0.00993467,0.160356,0.0185273,0.0972,0.0218567,0.0358574,0,0.023638,1,1,...,1,1,1,1,1,1,1,1,1,0.0685901
stddev_bias,0.051584,0.0654104,0.0621333,0.065248,0.0216307,0.0176823,0.00261235,0.0322701,0.0250266,0.0896132,...,0.0250093,0.0815664,0.020129,0.0658972,0.0633467,0.0201246,0.0896132,0.0705667,0.118725,0.00497532
score,98.462,94.3558,97.9835,95.9388,98.9128,98.6615,99.9347,98.6023,74.3743,72.7597,...,74.3748,72.9608,74.4968,73.3526,73.4163,74.4969,72.7597,73.2358,72.0319,98.1609


In [9]:
import pandas as pd
cmp_df = cmp_df[['X_20','X_80']]
score_list = cmp_df.loc['score'].to_list()
while -1 in score_list:
    score_list.remove(-1)
score_list
import numpy as np
np.mean(score_list)

73.03549459195969

In [3]:
km_conf = {
    "compare_job_id": job_id,
    "type": "csv",
    "method": "kmeans",
    "key": "y",
    "selected_features_list": ['X_20','X_80']
}
# 提交评估任务
km_evaluation_job = submitter.submit_evaluation_job(**km_conf)
print(km_evaluation_job.to_dict())
km_evaluation_job_id = km_evaluation_job.job_id
km_evaluation_job_id

2021-05-29 16:47:04,777 - INFO - request: http://localhost:8000/v1/evaluation/job/ with data {'method': 'kmeans', 'type': 'csv', 'compare_job_id': 10085, 'key': 'y', 'selected_features_list': ['X_20', 'X_80']}


{'code': 0, 'msg': '', 'data': {'job_id': 50022}}


50022

In [6]:
# 查看各属性评估得分
km_evaluation_job_data = submitter.get_evaluation_job_details(job_id=km_evaluation_job_id)
km_score = km_evaluation_job_data.result
km_score

2021-05-29 16:47:25,262 - INFO - request: http://localhost:8000/v1/query/evaluation/job/ with data {'job_id': 50022}


{'score': 99.0, 'accuracy': 0.98989898989899, 'centers_result': 99}

In [20]:
# 简单的读取抽样之后的文件
from pyspark.sql import SparkSession
from sparksampling.config import SPARK_CONF

conf = SPARK_CONF
spark = SparkSession.builder.config(conf=conf).getOrCreate()
df = spark.read.csv(dataset_uri, header=True).toPandas()
# 可以在这后面做数据分析，或试试看下面的统计、评估功能
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()

train_y = df['y']
train_X = df[['X_20','X_80']]
# train_X = df.drop(["# id"], axis=1)
model.fit(train_X,train_y)
tsdf = spark.read.csv(dataset_uri, header=True)
tdf = tsdf.toPandas()

test_y = tdf['y']
test_X = tdf[train_X.columns]
# test_X = test_X[feature_list]
pred_y = model.predict(test_X)
# data analyse here
from sklearn.metrics import classification_report
print(classification_report(y_true=test_y, y_pred=pred_y))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       517
           1       1.00      1.00      1.00       483

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000

